Classification showing and telling with pytorch_transformer(BERT)

In [20]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [21]:
import pandas as pd
from pandas import DataFrame

import numpy as np


data = pd.read_csv('data/showingTelling_csv.csv', delimiter = ',', encoding="ISO-8859-1")

data.index.name = "index"
data.columns = ["label", "text"]
data

,label,text
index,,
0,0,John was sad to see his girlfriend leave.
1,0,The house was creepy.
2,0,I heard footsteps creeping behind me and it ma...
3,0,She was my best friend. I could tell her almos...
4,0,She hated it there because it smelled bad.
...,...,...
254,1,"Her hand reached for the massive, iron door ha..."
255,1,The way the door decisively slammed behind her...
256,1,Dust coated every last surface. He ran his fin...


train_df = pd.read_csv('./nsmc/showingTelling_csv.csv', sep='\t', encoding='utf-16')
test_df = pd.read_csv('./nsmc/showingTelling_csv.txt', sep='\t', encoding='utf-16')

In [22]:
!pwd

/Users/kimkwangil/Project/01EssayFitAI/showing_telling


In [23]:
from sklearn.model_selection import train_test_split

def shuffle(df, n=1, axis=0): #데이터가 1111 0000이라 잘 섞어주자. 
    df = data.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

shuffle(data)

train_df, test_df = train_test_split(data, test_size=0.2)
print(len(train))
print(len(test))

207
52


In [24]:
train_df

,label,text
index,,
21,0,was very painful.
154,1,barely make it to my bus on time. Once in the ...
198,1,Aaron stepped out into the bright summer sun w...
180,1,"A sheet of paper, blank on the upper side, but..."
115,1,The faint scent of stale cigarette smoke met h...
...,...,...
179,1,Katie picks at the bottom and it comes away.
145,1,"Bright-red ribbons, dimly-lit candles, and a g..."
182,1,Roger had been working on the crossword puzzle...


In [25]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=1, random_state=999)
test_df = test_df.sample(frac=1, random_state=999)

/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
train_df

,label,text
index,,
109,0,"And he thought about how lovely Emily was, ser..."
248,1,While she was out of the room he looked at the...
80,0,"James was tall and attractive to women, being ..."
188,1,Brenda slammed his dresser drawer shut and spu...
236,1,Lucy Knight mince-stepped around clumps of hor...
...,...,...
231,1,The train disappeared over the horizon; all th...
132,1,"The lights suddenly went out. In the darkness,..."
126,1,"championships, sings lead vocals in a rock ban..."


In [27]:
train_df.iloc[0, 1]

'And he thought about how lovely Emily was, serving him all the good desert and drinks while wearing a summer dress.'

In [28]:
train_df.shape

(207, 2)

In [29]:
class STDataset(Dataset):
    ''' Showing Telling Corpus Dataset '''
    def __init__(self, df): #데이터 전처리
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = int(self.df.iloc[idx, 0])
        return text, label

In [30]:
ST_train_dataset = STDataset(train_df)
train_loader = DataLoader(ST_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [31]:
ST_train_dataset.__getitem__(7)

('run down the stairs, let my dog out, and inhale my breakfast. I scramble for my books and jacket, then race out the front door. I',
 1)

In [32]:
ST_train_dataset

In [33]:
train_loader

In [34]:
import torch

#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [35]:
for text, label in train_loader :
    print("text:",text)
    print("label:",label)

text: ('Joey hadn\x81ft seen his father for a long, long time _ not since he was in kindergarten. He couldn\x81ft remember his father\x81fs face very well. But at night, he could still hear him say, \x81gGood night, Joey. I love you.\x81h', 'Katie picks at the bottom and it comes away.')
label: tensor([1, 1])
text: ('October arrived, spreading a damp chill over the grounds and into the castle. Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. Ginny Weasley, who had been looking pale, was bullied into taking some by Percy. The steam pouring from under her vivid hair gave the impression that her whole head was on fire.', 'The night was cold and moonlit. The sleigh moved fast through the forest.')
label: tensor([1, 0])
text: ('The little girl looked in the window.', 'Two hockey players take unsuccessful swipes at the puck.')
label

In [ ]:
#working properly but recomment using google colab!

optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1


/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 